# Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Time series
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose

# ML
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb


import warnings
warnings.filterwarnings('ignore')

# Configurations
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Loading data

In [3]:
# Load sales and market data
sales_df = pd.read_csv('../Data/Case2_Sales data.csv', sep=";")
market_df = pd.read_excel('/Users/diogocarvalho/Documents/GitHub/Case-2--Siemens-Sales-Forecast/Data/Case2_Market data.xlsx', header=[0,1,2])

# Preprocessing sales data

> Sales_EUR is object and must be numeric

In [4]:
# fix the type of Sales_EUR column
sales_df['Sales_EUR'] = (
    sales_df['Sales_EUR']
    .replace(',','.', regex=True)        
    .replace(r'[^0-9\.\-]', '', regex=True) 
    .pipe(pd.to_numeric, errors='coerce')
)

In [5]:
sales_df['Sales_EUR'].dtype

dtype('float64')

> DATE is object, and must be converted to datetime

In [6]:
# Convert the DATE column to datetime
sales_df['DATE'] = pd.to_datetime(sales_df['DATE'], format='%d.%m.%Y')

In [7]:
# Aggregate to monthly level
sales_df['YearMonth'] = sales_df['DATE'].dt.to_period('M')
monthly_sales = sales_df.groupby(['YearMonth', 'Mapped_GCK'])['Sales_EUR'].sum().reset_index()

# Convert YearMonth to datetime
monthly_sales['YearMonth'] = monthly_sales['YearMonth'].dt.to_timestamp()

In [8]:
# make DATE index of sales_df
# sales_df = sales_df.set_index('DATE')

In [9]:
sales_df.head()

,DATE,Mapped_GCK,Sales_EUR,YearMonth
0,2018-10-01,#1,0.0,2018-10
1,2018-10-02,#1,0.0,2018-10
2,2018-10-03,#1,0.0,2018-10
3,2018-10-04,#1,0.0,2018-10
4,2018-10-05,#1,0.0,2018-10


> Aggregating sales per product based on month and year

Since we will be doing monthly analysis, it's no use for us to have the data in days.

In [10]:
# Aggregate sales data by month for each product
monthly_sales_data = sales_df.groupby(['YearMonth', 'Mapped_GCK'], as_index=False).agg({
    'Sales_EUR': 'sum'  # Sum sales revenue
})

In [11]:
# Display the first few rows to check the transformation
monthly_sales_data.head(25)

,YearMonth,Mapped_GCK,Sales_EUR
0,2018-10,#1,36098918.79
1,2018-10,#11,1021303.50
2,2018-10,#12,28686.33
3,2018-10,#13,27666.10
4,2018-10,#14,5770.00
5,2018-10,#16,333196.87
6,2018-10,#20,4563.14
7,2018-10,#3,8089465.96
8,2018-10,#36,6474.60
9,2018-10,#4,397760.69


In [12]:
monthly_sales_data.shape

(602, 3)

### Separate products into different datasets

We will predcit procuts individually, so separating them will be useful

In [13]:
# replace the "#" with "Product" in the product names
monthly_sales_data['Mapped_GCK'] = monthly_sales_data['Mapped_GCK'].str.replace('#', 'Product')

In [14]:
# Get the unique products
unique_products = monthly_sales_data['Mapped_GCK' ].unique()
unique_products

array(['Product1', 'Product11', 'Product12', 'Product13', 'Product14',
       'Product16', 'Product20', 'Product3', 'Product36', 'Product4',
       'Product5', 'Product6', 'Product8', 'Product9'], dtype=object)

In [15]:
# Create a dictionary to store the DataFrames
product_dataframes = {product: monthly_sales_data[monthly_sales_data['Mapped_GCK'] == product] for product in unique_products}

In [16]:
# Loop through each product and assign it to a dynamically named variable
for i, product in enumerate(product_dataframes.keys(), start=1):
    globals()[f"product{i}"] = product_dataframes[product]

In [17]:
product1.head()

,YearMonth,Mapped_GCK,Sales_EUR
0,2018-10,Product1,36098918.79
14,2018-11,Product1,5140760.00
28,2018-12,Product1,37889612.12
42,2019-01,Product1,27728148.35
56,2019-02,Product1,34793163.53


In [18]:
# Check the shape of each product DataFrame
for i in range(1, len(product_dataframes) + 1):
    print(f"product{i} shape: {globals()[f'product{i}'].shape}")

product1 shape: (43, 3)
product2 shape: (43, 3)
product3 shape: (43, 3)
product4 shape: (43, 3)
product5 shape: (43, 3)
product6 shape: (43, 3)
product7 shape: (43, 3)
product8 shape: (43, 3)
product9 shape: (43, 3)
product10 shape: (43, 3)
product11 shape: (43, 3)
product12 shape: (43, 3)
product13 shape: (43, 3)
product14 shape: (43, 3)


In [19]:
for i in range(1, 15):
    df = globals()[f'product{i}']
    
    # Convert Period to Timestamp (datetime type)
    df['DATE'] = df['YearMonth'].dt.to_timestamp()
    
    globals()[f'product{i}'] = df

In [20]:
# check if the conversion was successful
# for i in range(1, len(product_dataframes) + 1):
#     print(f"product{i} first 5 rows:{globals()[f'product{i}'].info()}")

# Preprocessing market data

In [21]:
market_df.head()

Unnamed: 0_level_0  \
  Index 2010=100 (if not otherwise noted)   
                                     date   
0                                  2004m2   
1                                  2004m3   
2                                  2004m4   
3                                  2004m5   
4                                  2004m6   

                                     China  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO156   
0                                16.940704   
1                                23.711852   
2                                24.435235   
3                                23.708115   
4                                27.009138   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP156   
0                               16.940704   
1                               23.711852   
2                               24.435235   
3                               23.708115   
4                               27.009138   

                                    France  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO250   
0                               112.091273   
1                               136.327976   
2                               117.791806   
3                               109.002541   
4                               133.785737   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP250   
0                               83.458866   
1                              106.168192   
2                               92.007646   
3                               85.696486   
4                              106.641482   

                                   Germany  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO276   
0                                82.623037   
1                               100.556582   
2                                89.653203   
3                                86.880571   
4                                99.010814   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP276   
0                               79.452532   
1                               97.012918   
2                               84.932358   
3                               82.372794   
4                               95.108740   

                                     Italy  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO380   
0                               124.289603   
1                               143.411662   
2                               129.083828   
3                               135.590391   
4                               136.424935   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP380   
0                               86.560493   
1                              106.344544   
2                               95.579673   
3                              100.087039   
4                              110.889719   

                                     Japan  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO392   
0                               109.334010   
1                               140.884616   
2                               105.853579   
3                               101.864777   
4                               120.332920   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP392   
0                              110.495272   
1                              144.686166   
2                              102.655769   
3                              100.305285   
4                              119.616380   

            

In [22]:
# drop the second header row
market_df.columns = market_df.columns.droplevel(1)

In [23]:
# Join the header levels
market_df.columns = [
    '_'.join([str(level) for level in col_tuple]).strip()
    for col_tuple in market_df.columns
]


In [24]:
# change first column name to "DATE"
market_df.rename(columns={'Unnamed: 0_level_0_date': 'DATE'}, inplace=True)

In [25]:
# Replace the "m" with "-" in DATE column of market_df
market_df['DATE'] = market_df['DATE'].str.replace('m', '-')

In [26]:
# Convert the DATE column to datetime
market_df['DATE'] = pd.to_datetime(market_df['DATE'], infer_datetime_format=True)

In [27]:
# Create the YearMonth column, like done before in the sales_df
market_df['YearMonth'] = market_df['DATE'].dt.to_period('M')

In [28]:
# Convert YearMonth to match sales_df
market_df['YearMonth'] = market_df['YearMonth'].dt.strftime('%Y-%m')

# Merge market data

In [29]:
for i in range(1, 15):
    df = globals()[f'product{i}']
    
    # Ensure both sides have datetime-type DATE columns (just in case)
    df['DATE'] = pd.to_datetime(df['DATE'])
    market_df['DATE'] = pd.to_datetime(market_df['DATE'])

    # Merge market_df into the product sales data
    merged_df = pd.merge(df, market_df, on='DATE', how='left')

    # Optional: sort by date (and product, if exists)
    merged_df = merged_df.sort_values(['DATE']).reset_index(drop=True)
    
    # Save back
    globals()[f'product{i}'] = merged_df

In [30]:
# # Ensure macro data is monthly
# market_df['YearMonth'] = market_df['DATE'].dt.to_period('M').dt.to_timestamp()

# # Merge with sales
# full_df = pd.merge(monthly_sales, market_df, on='YearMonth', how='left')

# # Sort for time series modeling
# full_df = full_df.sort_values(['Mapped_GCK', 'YearMonth']).reset_index(drop=True)

In [31]:
# check for rows where YearMonth_x is not equal to YearMonth_y
for i in range(1, 15):
    df = globals()[f'product{i}']
    print(f"product{i} shape: {df[df['YearMonth_x'] != df['YearMonth_y']].shape}")

product1 shape: (0, 52)
product2 shape: (0, 52)
product3 shape: (0, 52)
product4 shape: (0, 52)
product5 shape: (0, 52)
product6 shape: (0, 52)
product7 shape: (0, 52)
product8 shape: (0, 52)
product9 shape: (0, 52)
product10 shape: (0, 52)
product11 shape: (0, 52)
product12 shape: (0, 52)
product13 shape: (0, 52)
product14 shape: (0, 52)


In [32]:
# drop the YearMonth_y column in all product DataFrames because it's same as YearMonth_x
for i in range(1, 15):
    df = globals()[f'product{i}']
    df.drop(columns='YearMonth_y', inplace=True)
    globals()[f'product{i}'] = df

In [33]:
# change the name of YearMonth_x to YearMonth
for i in range(1, 15):
    df = globals()[f'product{i}']
    df.rename(columns={'YearMonth_x': 'YearMonth'}, inplace=True)
    globals()[f'product{i}'] = df

In [34]:
# Make DATE the index of the DataFrame
for i in range(1, 15):
    df = globals()[f'product{i}']
    df = df.set_index('DATE')
    globals()[f'product{i}'] = df

In [35]:
product1.columns

Index(['YearMonth', 'Mapped_GCK', 'Sales_EUR', 'China_MAB_ELE_PRO156',
       'China_MAB_ELE_SHP156', 'France_MAB_ELE_PRO250',
       'France_MAB_ELE_SHP250', 'Germany_MAB_ELE_PRO276',
       'Germany_MAB_ELE_SHP276', 'Italy_MAB_ELE_PRO380',
       'Italy_MAB_ELE_SHP380', 'Japan_MAB_ELE_PRO392', 'Japan_MAB_ELE_SHP392',
       'Switzerland_MAB_ELE_PRO756', 'Switzerland_MAB_ELE_SHP756',
       'United Kingdom_MAB_ELE_PRO826', 'United Kingdom_MAB_ELE_SHP826',
       'United States_MAB_ELE_PRO840', 'United States_MAB_ELE_SHP840',
       'Europe_MAB_ELE_PRO1100', 'Europe_MAB_ELE_SHP1100',
       'Europe_RohiBASEMET1000_org', 'Europe_RohiENERGY1000_org',
       'Europe_RohiMETMIN1000_org', 'Europe_RohiNATGAS1000_org',
       'Europe_RohCRUDE_PETRO1000_org', 'Europe_RohCOPPER1000_org',
       'Europe_WKLWEUR840_org', 'Producer Prices_PRI27840_org',
       'Producer Prices_PRI27826_org', 'Producer Prices_PRI27380_org',
       'Producer Prices_PRI27250_org', 'Producer Prices_PRI27276_org',
    

In [36]:
product4.shape

(43, 50)

In [37]:
# check if the change was successful
for i in range(1, len(product_dataframes) + 1):
     print(f"product{i} shape is:{globals()[f'product{i}'].shape}")

product1 shape is:(43, 50)
product2 shape is:(43, 50)
product3 shape is:(43, 50)
product4 shape is:(43, 50)
product5 shape is:(43, 50)
product6 shape is:(43, 50)
product7 shape is:(43, 50)
product8 shape is:(43, 50)
product9 shape is:(43, 50)
product10 shape is:(43, 50)
product11 shape is:(43, 50)
product12 shape is:(43, 50)
product13 shape is:(43, 50)
product14 shape is:(43, 50)


In [38]:
# check if the change was successful
for i in range(1, len(product_dataframes) + 1):
     print(f"product{i} first 5 rows:{globals()[f'product{i}'].head()}")

product1 first 5 rows:           YearMonth Mapped_GCK    Sales_EUR  China_MAB_ELE_PRO156  \
DATE                                                                 
2018-10-01   2018-10   Product1  36098918.79            211.955755   
2018-11-01   2018-11   Product1   5140760.00            220.519655   
2018-12-01   2018-12   Product1  37889612.12            241.846854   
2019-01-01   2019-01   Product1  27728148.35            175.668147   
2019-02-01   2019-02   Product1  34793163.53            175.668147   

            China_MAB_ELE_SHP156  France_MAB_ELE_PRO250  \
DATE                                                      
2018-10-01            211.955755             108.280608   
2018-11-01            220.519655              99.636911   
2018-12-01            241.846854              94.690312   
2019-01-01            175.668147              90.143775   
2019-02-01            175.668147              92.551521   

            France_MAB_ELE_SHP250  Germany_MAB_ELE_PRO276  \
DATE        

# Train-test split

> Doing the split before feature engineering **prevents data leakage**, which could give the model an unrealistic peek into the future.

We split the data in Jannuary 2022, leaving 39 months for training and 4 for testing (~90-10 split):
- train_product1 to train_product14 – data until December 2021
- test_product1 to test_product14 – data from Jannuary 2022 onward

In [39]:
# Define the split date
split_date = pd.to_datetime("2021-12-31")

# Loop through each product DataFrame
for i in range(1, 15):
    df = globals()[f'product{i}']

    # Ensure index is datetime and sorted
    df = df.sort_index()

    # Perform the split using the index
    train_df = df[df.index <= split_date].copy()
    test_df = df[df.index > split_date].copy()

    # Save them back to memory
    globals()[f'train_product{i}'] = train_df
    globals()[f'test_product{i}'] = test_df

In [40]:
train_product1.shape

(39, 50)

In [41]:
test_product1.shape

(4, 50)

In [42]:
test_product1.head()

,YearMonth,Mapped_GCK,Sales_EUR,China_MAB_ELE_PRO156,China_MAB_ELE_SHP156,France_MAB_ELE_PRO250,France_MAB_ELE_SHP250,Germany_MAB_ELE_PRO276,Germany_MAB_ELE_SHP276,Italy_MAB_ELE_PRO380,Italy_MAB_ELE_SHP380,Japan_MAB_ELE_PRO392,Japan_MAB_ELE_SHP392,Switzerland_MAB_ELE_PRO756,Switzerland_MAB_ELE_SHP756,United Kingdom_MAB_ELE_PRO826,United Kingdom_MAB_ELE_SHP826,United States_MAB_ELE_PRO840,United States_MAB_ELE_SHP840,Europe_MAB_ELE_PRO1100,Europe_MAB_ELE_SHP1100,Europe_RohiBASEMET1000_org,Europe_RohiENERGY1000_org,Europe_RohiMETMIN1000_org,Europe_RohiNATGAS1000_org,Europe_RohCRUDE_PETRO1000_org,Europe_RohCOPPER1000_org,Europe_WKLWEUR840_org,Producer Prices_PRI27840_org,Producer Prices_PRI27826_org,Producer Prices_PRI27380_org,Producer Prices_PRI27250_org,Producer Prices_PRI27276_org,Producer Prices_PRI27156_org,production index_PRO28840_org,production index_PRO281000_org,production index_PRO28756_org,production index_PRO28826_org,production index_PRO28380_org,production index_PRO28392_org,production index_PRO28250_org,production index_PRO28276_org,production index_PRO27840_org,production index_PRO271000_org,production index_PRO27756_org,production index_PRO27826_org,production index_PRO27380_org,production index_PRO27392_org,production index_PRO27250_org,production index_PRO27276_org
DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-01-01,2022-01,Product1,37942942.06,235.956129,235.956129,85.743503,108.156320,94.550610,120.353403,86.851008,101.258277,110.460181,110.823532,103.499260,101.701570,95.003541,NaN,111.052133,129.565798,103.199827,120.338095,133.219393,121.309886,125.229641,196.911140,106.173052,129.829146,1.1314,131.628510,NaN,115.390617,111.037476,117.853386,98.280171,110.894371,117.489883,100.305236,85.444170,92.292313,117.861377,90.558372,92.343117,111.364670,122.236023,108.999212,112.324119,74.355736,95.369065,77.944954,98.599052
2022-02-01,2022-02,Product1,36293076.33,235.956129,235.956129,90.603540,117.715770,103.987916,129.383676,106.583758,120.956538,117.879631,118.300232,100.294492,98.583952,98.458412,NaN,116.336327,138.560330,113.500635,131.500126,138.905572,131.273215,131.176501,197.523679,118.348203,131.963648,1.1342,133.342178,NaN,116.431107,112.057098,118.905647,98.714158,117.168167,124.627762,98.332942,89.021378,113.290565,124.710859,97.766502,102.820961,114.688400,127.373421,103.672183,115.557330,91.182419,103.950687,79.001831,106.128059
2022-03-01,2022-03,Product1,40220407.03,329.413367,329.413367,107.843548,136.858720,121.308119,151.201314,124.637966,153.645142,152.000561,156.400634,97.089723,95.466333,121.993915,NaN,117.654038,165.926217,133.133010,158.055622,149.890871,163.186834,141.283339,271.079906,142.200872,135.782207,1.1019,136.153778,NaN,117.471596,112.362991,119.852684,99.021554,118.910912,149.375229,96.360648,109.155949,134.288818,160.954233,114.720810,122.049515,115.164093,152.452942,98.345154,145.254965,102.475998,133.743932,96.704582,119.948433
2022-04-01,2022-04,Product1,41053203.12,267.373145,267.373145,87.698110,116.528738,99.522205,127.022869,103.556690,128.733305,114.262328,115.012049,NaN,NaN,95.266502,NaN,116.961047,NaN,112.902215,134.935510,146.090998,153.188945,138.094143,243.436030,130.835430,134.859685,1.0819,137.531616,NaN,118.408043,113.280655,121.220627,98.857087,119.385483,128.285706,NaN,84.728728,111.090744,120.098810,91.979698,98.675873,112.158089,134.843353,NaN,114.359844,86.255684,102.361680,80.763306,101.074341


# Feature Engineering

## Lag features for Sales_EUR

`Lag features` answer the question "what were the sales X months ago?" - we are using lag with X=[1,3,12] (1 month, 1 trimester, 1 year)

In [43]:
# include recent sales for each product
for i in range(1, 15):
    df = globals()[f'train_product{i}']
    df['Sales_Lag_1'] = df['Sales_EUR'].shift(1)
    df['Sales_Lag_3'] = df['Sales_EUR'].shift(3)
    df['Sales_Lag_12'] = df['Sales_EUR'].shift(12)
    globals()[f'train_product{i}'] = df

`Rolling mean(moving average)`capture short-term trends or smooths out noise. For example, April 2021, it averages Jan–Mar 2021 (but shifted 1 step so we're not using the current month’s value).


In [44]:
## Include rolling mean for each product
for i in range(1, 15):
    df = globals()[f'train_product{i}']
    df['Rolling_Mean_3'] = df['Sales_EUR'].shift(1).rolling(window=3).mean() # mean of the last 3 months
    df['Rolling_Mean_6'] = df['Sales_EUR'].shift(1).rolling(window=6).mean() # mean of the last 6 months
    globals()[f'train_product{i}'] = df

## Temporal features

We are creating columns to keep the month, and the quarter, as it may help identify seasonal trends

In [45]:
# create a month and a quarter column
for i in range(1, 15):
    df = globals()[f'train_product{i}']
    df['Month'] = df.index.month
    df['Quarter'] = df.index.quarter
    globals()[f'train_product{i}'] = df

## Lag features for Macro Variables

In [46]:
# Loop over all 14 product DataFrames
for i in range(1, 15):
    df = globals()[f'train_product{i}']
    
    # Identify macro columns (exclude sales/date/product ID columns)
    macro_cols = [col for col in df.columns if col not in [
        'YearMonth', 'Mapped_GCK', 'Sales_EUR', 'DATE'
    ] and not col.startswith('Sales') and not col in ['Month', 'Quarter', 'Product_ID']]
    
    # Create lag and delta features
    for col in macro_cols:
        df[f'{col}_Lag1'] = df[col].shift(1)
        df[f'{col}_Delta'] = df[col] - df[col].shift(1)
    
    # Save back to global scope
    globals()[f'train_product{i}'] = df

# Feature selection

## Feature correlation analysis

After analysing the feature correlation to Sales_EUR in each product, **we decided to keep only the best 5 for each product**, in order to keep the model light. 

`Note:` with more resources (time & computational power), we could keep more features a maybe achieve bettter results.

In [47]:
for i in range(1, 15):
    df = globals()[f'train_product{i}']
    
    # Drop any rows with NaNs just to be safe
    df_clean = df.dropna()

    # Compute correlations with the target variable
    correlation = df_clean.corr(numeric_only=True)['Sales_EUR'].drop('Sales_EUR').abs().sort_values(ascending=False)

    # Get top 5 correlated features
    top_features = correlation.head(5)

    print(f"\nTop 5 Correlated Features for train_product{i}:")
    print(top_features)


Top 5 Correlated Features for train_product1:
production index_PRO27826_org_Lag1     0.796230
Producer Prices_PRI27250_org_Lag1      0.782250
production index_PRO27276_org_Lag1     0.712610
United Kingdom_MAB_ELE_SHP826_Lag1     0.703399
production index_PRO271000_org_Lag1    0.697768
Name: Sales_EUR, dtype: float64

Top 5 Correlated Features for train_product2:
production index_PRO27392_org_Delta     0.773465
production index_PRO271000_org_Delta    0.771939
Japan_MAB_ELE_PRO392_Delta              0.769823
Japan_MAB_ELE_SHP392_Delta              0.768030
production index_PRO28392_org_Delta     0.755798
Name: Sales_EUR, dtype: float64

Top 5 Correlated Features for train_product3:
Producer Prices_PRI27826_org_Lag1    0.745661
Producer Prices_PRI27826_org         0.722219
Europe_RohCRUDE_PETRO1000_org        0.662970
Europe_RohiENERGY1000_org            0.662043
Europe_RohiENERGY1000_org_Lag1       0.629834
Name: Sales_EUR, dtype: float64

Top 5 Correlated Features for train_product4:
P

## Filter dataframes to only include the top 5 features

In [48]:
for i in range(1, 15):
    # Get train and test DataFrames
    train_df = globals()[f'train_product{i}']
    test_df = globals()[f'test_product{i}']

    # Drop NA rows from training set for correlation analysis
    train_clean = train_df.dropna()

    # Calculate correlation with Sales_EUR
    correlation = train_clean.corr(numeric_only=True)['Sales_EUR'].drop('Sales_EUR').abs().sort_values(ascending=False)
    top_5_features = correlation.head(5).index.tolist()

    # Include Sales_EUR in both sets
    selected_columns = top_5_features + ['Sales_EUR']

    # ✅ Only keep features present in both DataFrames
    available_columns = set(train_df.columns).intersection(test_df.columns)
    selected_columns = [col for col in selected_columns if col in available_columns]

    # Filter and drop NaNs
    train_filtered = train_df[selected_columns].dropna()
    test_filtered = test_df[selected_columns].dropna()

    # Create X and y
    if 'Sales_EUR' in train_filtered.columns:
        globals()[f'X_train_product{i}'] = train_filtered.drop(columns='Sales_EUR')
        globals()[f'y_train_product{i}'] = train_filtered['Sales_EUR']
    if 'Sales_EUR' in test_filtered.columns:
        globals()[f'X_test_product{i}'] = test_filtered.drop(columns='Sales_EUR')
        globals()[f'y_test_product{i}'] = test_filtered['Sales_EUR']

# TESTING - Model with XGBoost (ML Approach)

Why XGBoost?
- Handles non-linearity and interactions between features
- Works great with small-to-medium datasets
- Naturally includes feature importance
- Robust to multicollinearity

In [49]:
# To store models and RMSEs
xgb_models = {}
rmse_scores = {}

for i in range(1, 15):
    try:
        # Get training and test data
        X_train = globals()[f'X_train_product{i}']
        y_train = globals()[f'y_train_product{i}']
        X_test = globals()[f'X_test_product{i}']
        y_test = globals()[f'y_test_product{i}']

        # Train the model
        model = XGBRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=3,
            random_state=42,
            objective='reg:squarederror'
        )
        model.fit(X_train, y_train)

        # Predict and evaluate
        y_pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))

        # Store model and score
        xgb_models[f'product{i}'] = model
        rmse_scores[f'product{i}'] = rmse

        print(f"product{i} – RMSE: {rmse:.2f}")

    except Exception as e:
        print(f"product{i} – Error: {e}")

product1 – Error: list index out of range
product2 – Error: list index out of range
product3 – Error: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.
product4 – RMSE: 6224.11
product5 – Error: list index out of range
product6 – RMSE: 218568.18
product7 – RMSE: 3143.54
product8 – Error: list index out of range
product9 – RMSE: 7828.02
product10 – Error: list index out of range
product11 – Error: list index out of range
product12 – RMSE: 258980.72
product13 – Error: list index out of range
product14 – RMSE: 8465.40


# Visualize feature importance

In [50]:

xgb.plot_importance(xgb_models['product1'])
plt.title("Feature Importance – product1")
plt.show()

KeyError: 'product1'

In [221]:
# results = []

# # Loop through each product
# for gck in full_df['Mapped_GCK'].unique():
#     gck_df = full_df[full_df['Mapped_GCK'] == gck].dropna()
    
#     train = gck_df[gck_df['YearMonth'] < '2022-05-01']
#     test = gck_df[gck_df['YearMonth'] >= '2022-05-01']
    
#     features = [col for col in gck_df.columns if col not in ['Sales_EUR', 'Date', 'YearMonth', 'Mapped_GCK']]
    
#     model = XGBRegressor(n_estimators=100, learning_rate=0.1)
#     model.fit(train[features], train['Sales_EUR'])
    
#     preds = model.predict(test[features])
    
#     rmse = np.sqrt(mean_squared_error(test['Sales_EUR'], preds))
#     results.append((gck, rmse))
    
#     # Save predictions for submission
#     test['Predicted_Sales'] = preds
#     test[['YearMonth', 'Mapped_GCK', 'Predicted_Sales']].to_csv(f'predictions_{gck}.csv', index=False)

# Model with SARIMAX (Statistical Approach)

In [222]:
# # Example for 1 product
# gck = 'Product_X'
# gck_df = full_df[full_df['Mapped_GCK'] == gck].set_index('YearMonth')

# endog = gck_df['Sales_EUR']
# exog = gck_df[market_df.columns.difference(['Date', 'YearMonth'])]

# # Train/test split
# train_endog = endog[:'2022-04']
# test_endog = endog['2022-05':]

# train_exog = exog.loc[:'2022-04']
# test_exog = exog.loc['2022-05':]

# model = SARIMAX(train_endog, exog=train_exog, order=(1,1,1), seasonal_order=(1,1,1,12))
# sarimax_res = model.fit()

# forecast = sarimax_res.predict(start=test_endog.index[0], end=test_endog.index[-1], exog=test_exog)
# rmse = np.sqrt(mean_squared_error(test_endog, forecast))
# print(f"RMSE for {gck}: {rmse}")

# Evaluation

# Submission Formatting

In [223]:
# # Combine all test predictions into one file
# final_submission = pd.concat([pd.read_csv(f'predictions_{gck}.csv') for gck in full_df['Mapped_GCK'].unique()])
# final_submission.columns = ['Year Month', 'Mapped_GCK', 'Sales EUR']
# final_submission.to_csv("Submission_Template.csv", index=False)